In [1]:
#!/usr/bin/env python
import os
from sys import prefix
import nibabel as nib
from macbse import macbse

import SimpleITK as sitk


# Define paths
bse_model = "models/bias_field_correction_model_2024-03-02_22-29-46_epoch_9000.pth"

prefix = "sub-032196_ses-001_run-1_T1w_bst"
mri = f"{prefix}.nii.gz"

bseout = f"{prefix}.bse.nii.gz"
bfcout = f"{prefix}.bfc.nii.gz"
biasfield = f"{prefix}.bias.nii.gz"
maskfile = f"{prefix}.mask.nii.gz"
cerebrum_maskfile = f"{prefix}.cerebrum.mask.nii.gz"
pvcfile = f"{prefix}.pvc.frac.nii.gz"
pvc_label_file = f"{prefix}.pvc.label.nii.gz"
warped_air_atlas = f'{prefix}.warped.airatlas.nii.gz'
warped_air_labels = f'{prefix}.hemi.label.nii.gz'
air_atlas = "/home/ajoshi/Downloads/VERVET/brainsuite/VALiDATe12-airatlas/VALiDATe12-t1.airatlas.nii.gz"
air_atlas_labels = "/home/ajoshi/Downloads/VERVET/brainsuite/VALiDATe12-airatlas/VALiDATe12-t1.airatlas.label.nii.gz"

reg_mat = f"{prefix}.airatlas.mat"


In [2]:


macbse(mri, bseout, bse_model, maskfile, device="cuda")


# use SImpleITK to perform bias field correction
# Read the input image
inputImage = sitk.ReadImage(bseout)

# Set up for processing
maskImage = sitk.ReadImage(maskfile)
inputImage = sitk.Cast(inputImage, sitk.sitkFloat32)
maskImage = sitk.Cast(maskImage, sitk.sitkUInt8)


metatensor(0.9691, device='cuda:0') metatensor(0.0073, device='cuda:0')


In [3]:

# Apply the N4BiasFieldCorrection filter
corrector = sitk.N4BiasFieldCorrectionImageFilter()
corrector.SetMaximumNumberOfIterations([50] * 3)
corrector.SetConvergenceThreshold(1e-6)
corrector.SetBiasFieldFullWidthAtHalfMaximum(0.15)

# Execute the filter
outputImage = corrector.Execute(inputImage, maskImage)
log_bias_field = corrector.GetLogBiasFieldAsImage(inputImage)
bias_field = sitk.Exp(log_bias_field)

# Write the result
sitk.WriteImage(outputImage, bfcout)
sitk.WriteImage(log_bias_field, biasfield)


In [4]:

# do tissue classification in WM GM and CSF. Use FSL's FAST from the command line
# fsl5.0-fast -t 1 -n 3 -g -o sub-032196_ses-001_run-1_T1w.bfc.nii.gz sub-032196_ses-001_run-1_T1w.bfc.nii.gz

# do tissue classification in WM GM and CSF. Use FSL's FAST from the command line
os.system(f"fast -t 1 -n 3 -g -o {prefix} {bfcout}")



0

In [5]:

# Generate Isosurface from the white matter probability map
from calendar import c
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import nibabel as nib

# Load the tissue probability maps
gm = nib.load(f"{prefix}_pve_1.nii.gz").get_fdata()
wm = nib.load(f"{prefix}_pve_2.nii.gz").get_fdata()
csf = nib.load(f"{prefix}_pve_0.nii.gz").get_fdata()

affine = nib.load(f"{prefix}_pve_0.nii.gz").affine

csf_msk = np.double(csf > 0)
gm_msk = np.double(gm > 0)
wm_msk = np.double(wm > 0)

pvc_frac = 1 * csf_msk*(gm_msk==0) 
pvc_frac += (wm_msk==0)*gm_msk*(1+gm)
pvc_frac += wm_msk*(2+wm)


nib.save(nib.Nifti1Image(np.float32(pvc_frac), affine=affine), pvcfile)

nib.save(nib.Nifti1Image(np.uint8(pvc_frac), affine=affine), pvc_label_file)


In [6]:
cmd = f"flirt -in {air_atlas} -ref {bfcout} -out {warped_air_atlas} -omat {reg_mat}"
os.system(cmd)

cmd = f"flirt -in {air_atlas_labels} -ref {bfcout} -out {warped_air_labels} -applyxfm -init {reg_mat} -interp nearestneighbour"
os.system(cmd)


v = nib.load(warped_air_labels).get_fdata()
v = np.uint8(v)
nib.save(nib.Nifti1Image(v, affine=affine), warped_air_labels)


In [7]:
v=nib.load(warped_air_labels).get_fdata()
m = nib.load(maskfile).get_fdata()
m[(v==3) | (v == 4)] = 0
m = (m > 0.5)
nib.save(nib.Nifti1Image(255*np.uint8(m), affine=affine), cerebrum_maskfile)


In [8]:

# Threshold the tissue probability maps
threshold = 0.5
  gm[gm < threshold] = 0
  wm[wm < threshold] = 0
  csf[csf < threshold] = 0

# Generate isosurfaces
verts_gm, faces_gm, _, _ = measure.marching_cubes(gm, level=0.5)
verts_wm, faces_wm, _, _ = measure.marching_cubes(wm, level=0.5)
verts_csf, faces_csf, _, _ = measure.marching_cubes(csf, level=0.5)


# Plot the isosurfaces
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection="3d")

# Fancy indexing: `verts[faces]` to generate a collection of trisurf objects
mesh_gm = Poly3DCollection(verts_gm[faces_gm], alpha=0.1)
mesh_wm = Poly3DCollection(verts_wm[faces_wm], alpha=0.1)
mesh_csf = Poly3DCollection(verts_csf[faces_csf], alpha=0.1)

face_color = [0.5, 0.5, 1]
mesh_gm.set_facecolor(face_color)
mesh_wm.set_facecolor(face_color)
mesh_csf.set_facecolor(face_color)

ax.add_collection3d(mesh_gm)
ax.add_collection3d(mesh_wm)
ax.add_collection3d(mesh_csf)

ax.set_xlim(0, gm.shape[0])
ax.set_ylim(0, gm.shape[1])
ax.set_zlim(0, gm.shape[2])

plt.show()



IndentationError: unexpected indent (3978296984.py, line 3)

In [ ]:

from dfsio import readdfs, writedfs
import numpy as np

class s:
    pass

s.faces = faces_gm
s.vertices = verts_gm

writedfs("gm_isosurfaces.dfs", s)


s.faces = faces_wm
s.vertices = verts_wm

writedfs("wm_isosurfaces.dfs", s)


In [ ]:

# Save the isosurfaces as a DFS file
import nibabel as nib
from nibabel import freesurfer
import numpy as np

# Create a NIfTI image from the isosurfaces
data = np.zeros((256, 256, 256))
data[verts_gm[:, 0], verts_gm[:, 1], verts_gm[:, 2]] = 1
data[verts_wm[:, 0], verts_wm[:, 1], verts_wm[:, 2]] = 2
data[verts_csf[:, 0], verts_csf[:, 1], verts_csf[:, 2]] = 3

# Save the NIfTI image
img = nib.Nifti1Image(data, np.eye(4))
nib.save(img, "tissue_isosurfaces.nii.gz")


# Save the isosurfaces as a DFS file
freesurfer.write_geometry(
    "tissue_isosurfaces.dfs", verts_gm, faces_gm)

freesurfer.write_geometry(
    "tissue_isosurfaces.dfs", verts_wm, faces_wm)



In [ ]:
verts_gm

In [ ]:


# Save the isosurfaces as a DFS file
freesurfer.write_geometry(
    "tissue_isosurfaces.dfs", verts_gm, faces_gm, labels=np.ones(verts_gm.shape[0])
)
freesurfer.write_geometry(
    "tissue_isosurfaces.dfs", verts_wm, faces_wm, labels=np.ones(verts_wm.shape[0])
)


# Create output directory if not exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load T1 image
t1_img = nib.load(input_path)

# Skull stripping
skullstrip = BET()
skullstrip.inputs.in_file = input_path
skullstrip.inputs.out_file = os.path.join(output_dir, "skullstripped.nii.gz")
skullstrip.run()

# Tissue classification
tissue_class = FAST()
tissue_class.inputs.in_files = os.path.join(output_dir, "skullstripped.nii.gz")
tissue_class.inputs.out_basename = "tissue_classified"
tissue_class.run()

# Output tissue probability maps
tissue_prob_files = [
    os.path.join(output_dir, "tissue_classified_pve_0.nii.gz"),  # GM
    os.path.join(output_dir, "tissue_classified_pve_1.nii.gz"),  # WM
    os.path.join(output_dir, "tissue_classified_pve_2.nii.gz"),  # CSF
]

# Saving tissue probability maps as separate NIfTI files
for i, tissue_prob_file in enumerate(tissue_prob_files):
    tissue_prob_map = nib.load(tissue_prob_file)
    nib.save(tissue_prob_map, os.path.join(output_dir, f"tissue_classified_{i}.nii.gz"))

print("Processing complete. Output files saved in:", output_dir)